In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("VarunTelecomAnalysis").master("local[5]").getOrCreate()

25/06/12 09:47:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/12 09:47:19 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/06/12 09:47:19 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/06/12 09:47:19 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
25/06/12 09:47:19 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
25/06/12 09:47:19 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.
25/06/12 09:47:19 WARN Utils: Service 'SparkUI' could not bind on port 4045. Attempting port 4046.
25/06/12 09:47:19 WARN Utils: Service 'SparkUI' could not bind on port 4046. Attempting port 4047.
25/06/12 09:47:19 WARN Utils: Serv

In [5]:
call = spark.read.csv('call_records_synthetic.csv',sep=',', header=True, inferSchema=True)

In [8]:
cust = spark.read.csv('customers_synthetic.csv',sep=',', header=True, inferSchema=True)

In [9]:
call.show(4)

+---------+---------+-------------------+--------+---------+
|caller_id|callee_id|     call_timestamp|duration|call_type|
+---------+---------+-------------------+--------+---------+
|CUST01250|EXT490702|2024-04-11 04:37:30|     193| domestic|
|CUST01624|CUST04473|2024-06-13 01:11:37|     105| domestic|
|CUST04407|CUST04407|2024-07-10 08:47:57|     398| domestic|
|CUST02639|CUST01618|2024-05-17 10:55:51|     447| domestic|
+---------+---------+-------------------+--------+---------+
only showing top 4 rows



In [13]:
cust.show(5)

+----------+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+-----+
|customerID|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|MultipleLines|InternetService|MonthlyCharges|TotalCharges|Churn|
+----------+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+-----+
| CUST00001|  Male|            0|     No|        No|    16|         Yes|           No|            DSL|        108.22|     1724.13|   No|
| CUST00002|Female|            0|     No|       Yes|    57|          No|           No|            DSL|         73.26|     4169.46|  Yes|
| CUST00003|  Male|            0|    Yes|       Yes|    57|          No|           No|    Fiber optic|         79.43|     4536.19|   No|
| CUST00004|  Male|            1|    Yes|        No|    56|         Yes|           No|    Fiber optic|        119.01|     6657.85|  Yes|
| CUST00005|  Male|            0|     No|

In [10]:
call.printSchema()

root
 |-- caller_id: string (nullable = true)
 |-- callee_id: string (nullable = true)
 |-- call_timestamp: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- call_type: string (nullable = true)



In [12]:
cust.printSchema()

root
 |-- customerID: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- SeniorCitizen: integer (nullable = true)
 |-- Partner: string (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- tenure: integer (nullable = true)
 |-- PhoneService: string (nullable = true)
 |-- MultipleLines: string (nullable = true)
 |-- InternetService: string (nullable = true)
 |-- MonthlyCharges: double (nullable = true)
 |-- TotalCharges: double (nullable = true)
 |-- Churn: string (nullable = true)



In [23]:
call.describe().show()

+-------+---------+---------+-------------------+------------------+-------------+
|summary|caller_id|callee_id|     call_timestamp|          duration|    call_type|
+-------+---------+---------+-------------------+------------------+-------------+
|  count|   100000|   100000|             100000|            100000|       100000|
|   mean|     null|     null|               null|         299.75911|         null|
| stddev|     null|     null|               null|299.73074064429466|         null|
|    min|CUST00001|CUST00001|2024-01-01 00:01:18|                 0|     domestic|
|    max|CUST05000|EXT999969|2024-12-31 23:57:36|              3657|international|
+-------+---------+---------+-------------------+------------------+-------------+



In [24]:
from pyspark.sql.functions import *

In [22]:
call.groupBy(call.caller_id).sum().show()

+---------+-------------+
|caller_id|sum(duration)|
+---------+-------------+
|CUST03236|         6399|
|CUST02025|         7941|
|CUST04677|         8076|
|CUST00050|         5109|
|CUST02589|         3345|
|CUST00865|         5659|
|CUST02801|         5679|
|CUST03281|         8184|
|CUST04132|         3065|
|CUST01974|         5545|
|CUST00437|         3666|
|CUST02893|         5793|
|CUST00166|         5061|
|CUST00917|         6657|
|CUST03976|         5712|
|CUST02357|         5170|
|CUST03693|         6126|
|CUST02234|         6320|
|CUST03971|         3731|
|CUST03301|         3817|
+---------+-------------+
only showing top 20 rows



In [47]:
ctype = call.groupBy(call.call_type).sum("duration")

In [54]:
call.groupBy(call.call_type).agg({'*': 'count', 'duration': 'sum'}).show()

+-------------+-------------+--------+
|    call_type|sum(duration)|count(1)|
+-------------+-------------+--------+
|     domestic|     26961698|   89921|
|international|      3014213|   10079|
+-------------+-------------+--------+



In [58]:
call.printSchema()

root
 |-- caller_id: string (nullable = true)
 |-- callee_id: string (nullable = true)
 |-- call_timestamp: timestamp (nullable = true)
 |-- duration: integer (nullable = true)
 |-- call_type: string (nullable = true)



In [59]:
call = call.select(call.caller_id, call.callee_id, call.call_timestamp.cast('date'),call.duration,call.call_type)

In [85]:
month_duration = call.groupBy(month(call.call_timestamp)).sum("duration")

In [86]:
month_duration.show()

+---------------------+-------------+
|month(call_timestamp)|sum(duration)|
+---------------------+-------------+
|                   12|      2561212|
|                    1|      2534368|
|                    6|      2498837|
|                    3|      2558459|
|                    5|      2497744|
|                    9|      2462890|
|                    4|      2426105|
|                    8|      2521196|
|                    7|      2570048|
|                   10|      2511335|
|                   11|      2470201|
|                    2|      2363516|
+---------------------+-------------+



In [65]:
from pyspark.sql.functions import *
from pyspark.sql.window import *

In [77]:
win_dif = Window.orderBy(col('month(call_timestamp)').asc())

In [80]:
month_duration.withColumn('difference',  col('sum(duration)') - lead(col('sum(duration)')).over(win_dif)).show()

25/06/12 11:23:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+---------------------+-------------+----------+
|month(call_timestamp)|sum(duration)|difference|
+---------------------+-------------+----------+
|                    1|      2534368|    170852|
|                    2|      2363516|   -194943|
|                    3|      2558459|    132354|
|                    4|      2426105|    -71639|
|                    5|      2497744|     -1093|
|                    6|      2498837|    -71211|
|                    7|      2570048|     48852|
|                    8|      2521196|     58306|
|                    9|      2462890|    -48445|
|                   10|      2511335|     41134|
|                   11|      2470201|    -91011|
|                   12|      2561212|      null|
+---------------------+-------------+----------+



In [88]:
call = call.withColumn('months', month('call_timestamp'))

In [89]:
call.show(4)

+---------+---------+--------------+--------+---------+------+
|caller_id|callee_id|call_timestamp|duration|call_type|months|
+---------+---------+--------------+--------+---------+------+
|CUST01250|EXT490702|    2024-04-11|     193| domestic|     4|
|CUST01624|CUST04473|    2024-06-13|     105| domestic|     6|
|CUST04407|CUST04407|    2024-07-10|     398| domestic|     7|
|CUST02639|CUST01618|    2024-05-17|     447| domestic|     5|
+---------+---------+--------------+--------+---------+------+
only showing top 4 rows



In [90]:
call.write.format('orc').partitionBy('months').save('call_data_orc')